-sandbox
<img width = '500' src="https://databricks.com/wp-content/uploads/2021/06/db-pride-logo.svg" style="float: left: margin: 30px"/>        <img src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 10px"/>              

<br>
> # Bronze para Silver

<br>
## 06 - Balanças comenciais


- Quando os investidores compram ações, o saldo de caixa é reduzido. 
- Quando eles vendem, o saldo de caixa aumenta. 
- Também se aplicam taxas, em associação com o tipo de subscrição dos investidores 

Computar os impactos negativos e positivos no saldo de caixa

-sandbox
<h2 style="color:red">Informações:</h2>

<br>
- Objetivo:
   - o que será construindo: transformar e mesclar ainda mais as tabelas Silver
   - gerar novas métricas
   - por que é útil: fluxo de trabalho típico que gera métricas finais para negócios
- extrair expressões colunares para variáveis 
    - ajudará tremendamente na legibilidade do código

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Configurações úteis<br>

In [0]:
%run ./Includes/12-ClassroomSetup

Preparing the Python environment...

Defining courseware-specific utility methods...

Defining user-facing utility methods...

Defining custom variables for this lesson...

Initializing Databricks Academy's testing framework...

Initializing Databricks Academy's services for generating dynamic data...

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Cleaning up the learning environment...no actions taken.

WARNING: This notebook was tested on DBR 7.0, but we found DBR 7.3. 
 Using an untested DBR may yield unexpected results and/or various errors 
 Please update your cluster configuration and/or download a newer version of this course before proceeding.

The following variables were defined for you: 
 financeDataPath : dbfs:/mnt/training/finance-org 
 The location of the finance dataset used in this capstone project. 
 
 sourceDataPath : dbfs:/mnt/training/finance-org/solutions 
 The location of the specific datasets used in this capstone project as source data. 
 
 silverStockOrdersPath : dbfs:/mnt/training/finance-org/solutions/silver/stock_orders 
 dbfs:/mnt/training/finance-org/solutions/silver/stock_orders 
 
 silverInvestorsPath : dbfs:/mnt/training/finance-org/solutions/silver/investors 
 dbfs:/mnt/training/finance-org/solutions/silver/investors 
 
 feesPath : dbfs:/mnt/training/finance-org/solutions/fees.csv 
 dbfs:/mnt/training/finance-org/solutions/fees.csv 
 
 targetDirectory : dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/06_balancas_comerciais_psp/silver/stock_order_spend_balance 
 dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/06_balancas_comerciais_psp/silver/stock_order_spend_balance 
 The following database were created for you: 
 Now using the database identified by databaseName : 
 thomaz_thomazrossito_com_br_capstone_finance_06_balancas_comerciais_psp 
 This is a private, per-notebook, database used to provide isolation from other users and exercises. 
 All done!

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Etapas a serem concluídas:

<br>

- Atualmente, os pedidos de estoque não incluem taxas. As taxas estão mudando para cada tipo de assinatura. 
  - Os tipos de assinatura são registrados na tabela Investidores. 
  - Ler três itens para juntar, a fim de calcular o saldo de despesas através da coluna **`expens_balance`**:

  * Ler a tabela pedidos de estoque através do diretório  **`silverStockOrdersPath`** para um DataFrame e delete a coluna **`clicked_items`**
  * Ler a tabela silver de Investors através do diretório **`silverInvestorsPath`** para um DataFrame e selecione apenas os campos **`investor_id`** e **`subscription_id`**.
  * Ler os dados de taxas a partir do diretório **`feesPath`** para um DataFrame e filtre os registros onde **`product_id`** é igual a zero
  
- Junte as ordens de ações com investidores através dos campos **`investor`** e **`investor_id`**. 
  - Manter todos os campos das Ordens de Ações e apenas o campo **`subscription_id`** de Investidores.

- Junte o DataFrame anterior com taxas com **`subscription_id`**. 
  - Manter todas as colunas do DataFrame anterior e apenas o campo **`taxas`** do dataframe taxas.

- Após realizar essas junções, temos informações sobre quanto de taxa se aplica a cada transação em Pedidos de Estoque.
- Gere uma nova coluna chamada **`expens_balance`** com estas condições:
  * Se o valor **`type`** for "BUY", multiplique **`volume`** e **`price`**, reduza a **`taxa`** e converta o resultado para negativo. Segue a fórmula:
    - saldo_de_pesas = -1 x [(volume x preço) - taxas]
  * Se a condição não for atendida, não deve torná-la negativa:
    - gastar_equilíbrio = [(volume x preço) - taxas]

- Grave o DataFrame resultante no diretório **`targetDirectory`** como uma tabela Delta usando a opção de substituição.
- Retorne o DataFrame resultante.


<p> O esquema do DataFrame deve ser:
  
|name|type|
|---|---|
|investor|LongType|
|ordertime|LongType|
|price|DoubleType|
|ticker|StringType|
|transaction_id|LongType|
|type|StringType|
|volume|LongType|
|order_timestamp|TimestampType|
|year|IntegerType|
|month|IntegerType|
|day|IntegerType|
|dow|IntegerType|
|subscription_id|LongType|
|fees|IntegerType|
|spend_balance|DoubleType|

In [0]:
def func_balancaComeciais(spark, silverStockOrdersPath, silverInvestorsPath, feesPath, targetDirectory):
  
  
  from pyspark.sql.functions import col, when, lit
  

  # Ler a tabela de pedidos de estoque **`silverStockOrdersPath`** para um DataFrame e deleta a coluna **`clicked_items`**
  stockOrders = (spark.read
                      .format("delta")
                      .load(silverStockOrdersPath)
                      .drop("clicked_items"))
  
  
  # Ler a tabela silver de Investors **`silverInvestorsPath`** para um DataFrame 
  # Selecione apenas os campos **`investor_id`** e **`subscription_id`**
  investors = (spark.read
                    .format("delta")
                    .load(silverInvestorsPath)
                    .select("investor_id", 
                            "subscription_id"))
  
  
  # Ler os dados de taxas **`feesPath`** para um DataFrame e filtre os registros onde **`product_id`** é igual a zero
  taxas = (spark.read
                .option("inferSchema","true")
                .option("header","true")
                .csv(feesPath)
                .filter(col("product_id") == 0))
  

  # Junte as ordens de ações com investidores, campos **`investor`** e **`investor_id`**
  # Manter todos os campos das Ordens de Ações e apenas **`investor_id`** de Investidores
  join_res = (stockOrders.join(investors, 
                               on = (stockOrders["investor"] == investors["investor_id"]), 
                               how = "left")
                         .select(stockOrders["*"], 
                                 investors["subscription_id"]))
  
  
  # Junte o DataFrame anterior com taxas **`subscription_id`** 
  # Manter todas as colunas do DataFrame anterior e apenas o campo **`taxas`** das taxas
  # Gere uma nova coluna denominada **`expens_balance`** com as condições que foram fornecidas
  df = (join_res.join(taxas, 
                      on = "subscription_id", 
                      how = "left")
                .select(join_res["*"], 
                        taxas["fees"])
                .withColumn("spend_balance", 
                            when(
                                 col("type") == "BUY", (lit(-1) * col("volume") * col("price") - col("fees")))
                            .otherwise(col("volume") * col("price") - col("fees"))))
  

  # Grave o DataFrame resultante **`targetDirectory`** como uma tabela Delta usando a opção overwrite.
  df.write \
    .mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .save(targetDirectory) 
  
  
  # Retorna o DataFrame.
  return df

In [0]:
finalDF = func_balancaComeciais(spark, silverStockOrdersPath, silverInvestorsPath, feesPath, targetDirectory)

In [0]:
display(finalDF)

investor,ordertime,price,ticker,transaction_id,type,volume,order_timestamp,year,month,day,dow,subscription_id,fees,spend_balance
338,1567349092,111.73638548592248,ADSK,581346394,SELL,20,2019-09-01T14:44:52.000+0000,2019,9,1,1,0,15,2219.7277097184497
207,1567356992,111.37637070548257,ADSK,581346415,SELL,9,2019-09-01T16:56:32.000+0000,2019,9,1,1,1,10,992.3873363493432
754,1568115351,113.91432885327923,ADSK,581346711,BUY,2,2019-09-10T11:35:51.000+0000,2019,9,10,3,1,10,-237.82865770655846
646,1569343432,116.82051241667328,ADSK,581347261,SELL,29,2019-09-24T16:43:52.000+0000,2019,9,24,3,0,15,3372.7948600835252
62,1569678328,112.05261289152836,ADSK,581347397,BUY,14,2019-09-28T13:45:28.000+0000,2019,9,28,7,0,15,-1583.736580481397
1193,1569747600,106.49,ADSK,581347428,SELL,7,2019-09-29T09:00:00.000+0000,2019,9,29,1,0,15,730.43
1076,1567510483,56.45650077835717,AEE,581347573,BUY,12,2019-09-03T11:34:43.000+0000,2019,9,3,3,0,15,-692.478009340286
494,1568552768,55.83932111029493,AEE,581347952,SELL,3,2019-09-15T13:06:08.000+0000,2019,9,15,1,0,15,152.5179633308848
221,1568632889,56.132305630577925,AEE,581348015,SELL,26,2019-09-16T11:21:29.000+0000,2019,9,16,2,0,15,1444.439946395026
841,1569251971,56.43313528939689,AEE,581348267,SELL,3,2019-09-23T15:19:31.000+0000,2019,9,23,2,1,10,159.29940586819066


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Validação</h2>

In [0]:
realityCheck(func_balancaComeciais, spark, silverStockOrdersPath, silverInvestorsPath, feesPath, targetDirectory)

Points,Test,Result
1,Returns correct schema,
1,Returns DataFrame with correct number of rows,
1,Returns DataFrame with correct results,
1,Silver table in place,
1,Silver table has correct content,


## <img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Próximo <p>

> * [Cliques]($./07-Cliques)

-sandbox

<a href="http://www.apache.org/">Apache Software Foundation</a> <p>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a>